#### **Import Libraries**

In [ ]:
import os 
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import numpy as np 
import random 
import math
from itertools import chain
from IPython.display import display, Markdown
import textwrap
import tiktoken
import csv
import time 
import pandas as pd 
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
import transformers
from transformers import DataCollatorWithPadding
from llmft.train import EncoderTrainer, EarlyStopping
from llmft.metrics import compute_recall
from llmft.losses import FocalLoss
from llmft.utils import predict
import seaborn as sns 
import jax 
import jax.numpy as jnp 
import optax 
from rfp import MLP, Model, ModelParams
from rfp.utils import batch_matrix_with_padding
from rfp.losses import binary_cross_entropy, Supervised_Loss, mse
from rfp.train import Trainer

#### **Parameters**

In [ ]:
seed = 0 
np.random.seed(seed)
noise = True 
verbose = False 
sample_size = 1000
version = 3 

#### **Set Up Paths**

In [ ]:
data_csv = './../../../toy-data/exp2/data_{version}.csv'

#### **Set Up Plotting**

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import rcParams
rcParams['image.interpolation'] = 'nearest'
rcParams['image.cmap'] = 'viridis'
rcParams['axes.grid'] = False
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('seaborn-v0_8-dark-palette')

from matplotlib import font_manager 
locations = './../../../styles/Newsreader'
font_files = font_manager.findSystemFonts(fontpaths=locations)
print(locations)
print(font_files[0])
for f in font_files: 
    font_manager.fontManager.addfont(f)
plt.rcParams["font.family"] = "Newsreader"

#### **First Stage Function**

In [ ]:
def fstage(var0, var1, var2, var3, var4):
    return (1.0-var4)

#### **Read in Data**

In [ ]:
df = pd.read_csv(data_csv)
indices = np.random.choice(df.index, size=sample_size, replace=False)
df = df.loc[indices].reset_index(drop=True)
n = len(df)  # Get the number of rows in df
df['FStage_Value'] = df.apply(lambda row: fstage(row['Var0'], row['Var1'], row['Var2'], row['Var3'], row['Var4']), axis=1)
if noise: 
    df['FStage_Value'] = df['FStage_Value'].sample(frac=1).reset_index(drop=True)
df['Instrument'] = np.random.binomial(n=1, p=0.5, size=n)
df['Treatment'] = np.random.binomial(n=1, p= df['FStage_Value'] * df['Instrument'], size=n)
df['Outcome'] = df['Treatment'] + 0.1*np.random.normal(size=n)


#### **Plot Treatment Distribution**

In [ ]:
fig = plt.figure(dpi=300, tight_layout=True, figsize=(7, 4.5))
ax = plt.axes(facecolor=(.95, .96, .97))

# Plot customizations
for key in 'left', 'right', 'top':
    ax.spines[key].set_visible(False)
ax.text(0., 1.02, s='Count', transform=ax.transAxes, size=14)
ax.yaxis.set_tick_params(length=0)
ax.yaxis.grid(True, color='white', linewidth=2)
ax.set_axisbelow(True)
plt.hist(df['FStage_Value'], color='#36454F')
plt.xlim(0, 1)
plt.xlabel('Probability of Treatment Given Instrument', size=14)
plt.show()

#### **Preprocess Data**

In [ ]:
X = jnp.array(df[['Race','Gender', 'Rent', 'Health']].values)
# Feature Matrix
min_val = jnp.min(X, axis=0)
max_val = jnp.max(X, axis=0)
X_normalized = (X - min_val) / (max_val - min_val)
print(f"Feature shape: \t\t{X.shape}")

D = jnp.array(df['Treatment'].values.reshape(-1,1)) 
Z = jnp.array(df['Instrument'].values.reshape(-1,1))
inputs = jnp.hstack((X_normalized, Z))

#### **Hyperparameters**

In [ ]:
lr = 1e-3
epochs = 1000
nodes = 32
split_frac = 0.5

#### **Model Setup**

In [ ]:
features = inputs.shape[1] ; print(f"Number of Features: {features}")
mlp = MLP([nodes, nodes], jax.nn.relu)
final_activation_fn = jax.nn.sigmoid 
model = Model(mlp, final_activation_fn)
supervised_loss = Supervised_Loss(binary_cross_entropy, model.embellished_fwd_pass)
yuri = Trainer(supervised_loss, optax.sgd(learning_rate=lr, momentum=0.9), epochs)

#### **Train**

In [ ]:
key = jax.random.PRNGKey(0)

# Create Keys
key, subkey1, subkey2, subkey3, subkey4 = jax.random.split(key, 5)

# Initialize the parameters
params = ModelParams.init_fn(subkey1, mlp, features)

# Split Train and Validation Data
idx = jax.random.permutation(subkey2, inputs.shape[0])
is_in_train = jnp.isin(jnp.arange(inputs.shape[0]), idx[:int(split_frac*inputs.shape[0])])
is_in_val = jnp.isin(jnp.arange(inputs.shape[0]), idx[int(split_frac*inputs.shape[0]):])

# Train the model
params, opt_params, train_loss_history, val_loss_history = yuri.train_with_val(params, inputs, D, jnp.ones_like(D), is_in_train, is_in_val)

#### **Plot Losses**

In [ ]:
plt.plot(train_loss_history, label='Validation')
plt.plot(val_loss_history, label='Training')
plt.legend()
plt.show()

#### **Estimate**

In [ ]:
Dhat = np.array(model.fwd_pass(opt_params, inputs))
Dhat1 = np.array(model.fwd_pass(opt_params, jnp.hstack((X_normalized, jnp.ones_like(Z)))))
Dhat0 = np.array(model.fwd_pass(opt_params, jnp.hstack((X_normalized, jnp.zeros_like(Z)))))
residuals = Dhat - (0.5*Dhat1 + 0.5*Dhat0)
est = jnp.linalg.lstsq(jnp.hstack((residuals.reshape(-1,1), jnp.ones_like(residuals.reshape(-1,1)))), jnp.array(df['Outcome'].values.reshape(-1,1)))[0][0]
df_result = pd.DataFrame({'Estimate': est})

#### **Save Results**

In [ ]:
# Define the path for the CSV file
file_path = f'./../../../toy-data/exp2/results/nn_{version}_{sample_size}_{noise}.csv'

# Check if the file already exists
if not os.path.exists(file_path):
    # If the file does not exist, write with headers
    df_result[['Estimate']].to_csv(file_path, mode='w', header=True, index=False)
else:
    # If the file exists, append without headers
    df_result[['Estimate']].to_csv(file_path, mode='a', header=False, index=False)